# Evo-Bayes-Agent

In [1]:
from Ionomy import IonPanda, BitTA
from decouple import config
import pandas as pd
import pandas_ta as ta

In [2]:
# import seaborn as sns
# from statistics import mode
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import StandardScaler
# from sklearn.ensemble import RandomForestClassifier
# from sklearn import metrics
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import KFold
# import numpy as np

In [3]:
MARKET = 'usd-btc'
CURRENCY = 'btc'
BASE = 'usd'
TIME = 'hour'
initial_capital = 5_000
MAX_BUY = 0.1
MAX_SELL = 0.1
FEE_RATE = 0.01

window_size = 32
layer_size = 500
output_size = 3

In [4]:
bta = BitTA(config('TREX_KEY'), config('TREX_SECRET'))
ionpd = IonPanda(config('IONOMY_KEY'), config('IONOMY_SECRET'))
bta.update(CURRENCY, BASE, TIME)
df = bta.df

In [5]:
# ema_10 = df.ta.ema(length=10).sub(df['close']).divide(df['close']).rename('ema_10')
# ema_50 = df.ta.ema(length=50).sub(df['close']).divide(df['close']).rename('ema_50')
# ema_100 = df.ta.ema(length=100).sub(df['close']).divide(df['close']).rename('ema_100')
# ema_200 = df.ta.ema(length=200).sub(df['close']).divide(df['close']).rename('ema_200')
# true_range = df.ta.true_range().divide(df['close']).rename('true_range')

# pct_change = df['close'].pct_change().rename('pct_change')
# target = pct_change > 0

In [6]:
# features_df = pd.concat([pct_change, ema_10, ema_50, ema_100, ema_200, true_range], axis=1).dropna()

In [7]:
# best_dic = {'acc': 0}
# kf = KFold(n_splits=10, random_state=42, shuffle=True)

# for train_index, test_index in kf.split(features_df):
#     train, test = features_df.iloc[train_index], features_df.iloc[test_index]
#     train_target, test_target = target.iloc[train_index], target.iloc[test_index]
#     for i in range(1,50):
#         model = RandomForestClassifier(n_estimators=i, random_state=0)
#         model.fit(train, train_target)
#         pred = np.round(model.predict(test))
#         acc = accuracy_score(test_target, pred)
#         if acc > best_dic['acc']:
#             best_dic['acc'] = acc
#             best_dic['num_estimators'] = i

In [8]:
from neuroevolution import DemiChad

Using TensorFlow backend.


In [9]:
demichad = DemiChad(df)

In [ ]:
demichad.meta_evolve()

|   iter    |  target   |   cxpb    |   indpb   |    mu     |   mutpb   | networ... | popula... |   sigma   | tournsize | window... |
-------------------------------------------------------------------------------------------------------------------------------------
gen	nevals	f_avg   	f_max  
0  	9     	0.728995	8.38022
1  	9     	4.90529 	13.1994
2  	8     	8.57503 	24.9162
3  	9     	10.0796 	23.4639
4  	8     	7.56663 	26.3506
5  	8     	12.3508 	26.3506
6  	9     	2.73236 	14.1968
7  	9     	6.40774 	16.9277
8  	8     	8.28479 	16.9277
9  	8     	6.74031 	16.9277
10 	9     	7.02276 	16.9277
11 	8     	10.1299 	18.677 
12 	9     	13.3809 	19.9719
13 	9     	7.53968 	19.9719
14 	9     	17.0788 	25.2813
15 	8     	11.9527 	25.2813
16 	9     	10.7826 	24.4055
17 	9     	11.3364 	24.4055
18 	8     	11.6877 	24.4055
19 	9     	19.1105 	24.8745
20 	7     	12.2629 	24.4055
21 	9     	11.2632 	25.55  
22 	9     	15.6677 	26.4899
23 	8     	20.813  	27.6599
24 	8     	17.5859 	28.1944
25 	

In [ ]:
demichad.evolve(20)

In [ ]:
demichad.omega()

In [ ]:
demichad.omega_chad.sell_history

In [ ]:
demichad.omega_chad.sell_history.profit.sum()/len(demichad.omega_chad.sell_history)

In [ ]:
demichad.omega_chad.sell_history.profit.sum()

In [ ]:
demichad.plot()